The amplification factor in backward Lax-Wendroff is 
$$\epsilon(k,\sigma) = \left(1 + \frac{\sigma^2}{2} - \frac{3}{2}\sigma\right) - \left(\sigma^2 - 2 \sigma\right)e^{-\dot{\iota}kh} + \left(\frac{\sigma^2}{2} - \frac{\sigma}{2}\right)e^{-2\dot{\iota}kh} $$
We then put $-kh = \theta$ and define
$$\tilde{\epsilon}(\theta,\sigma) = \left(1 + \frac{\sigma^2}{2} - \frac{3}{2}\sigma\right) - \left(\sigma^2 - 2 \sigma\right)e^{\dot{\iota}\theta} + \left(\frac{\sigma^2}{2} - \frac{\sigma}{2}\right)e^{2\dot{\iota}\theta}$$
We'd then find $\sigma$ for which $$|\tilde{\epsilon}(\sigma,\theta)| \le 1, \quad \forall \theta \in [0,2\pi]$$

In [1]:
import numpy as np

In [29]:
#Defining amplification factor
def amp(theta,sigam):
    amp = (1.0 + sigma**2/2.0 - 1.5 * sigma) - (sigma**2 - 2.0*sigma)*np.exp(1j*theta) + (sigma**2/2.0 - sigma/2.0) * np.exp(2j*theta)
    return amp

In [50]:
theta_grid = np.linspace(0.0,2.0*np.pi, 100)
sigma_set = np.linspace(0.0,3.0,30)

In [51]:
for sigma in sigma_set:
    amplification = 0.0
    for theta in theta_grid:
        amplification = max(amplification,np.abs(amp(theta,sigma)))
    print("Amplification factor for sigma = ", sigma, " is ", amplification)

Amplification factor for sigma =  0.0  is  1.0
Amplification factor for sigma =  0.10344827586206896  is  1.0
Amplification factor for sigma =  0.20689655172413793  is  1.0000000000000002
Amplification factor for sigma =  0.3103448275862069  is  1.0
Amplification factor for sigma =  0.41379310344827586  is  1.0000000000000002
Amplification factor for sigma =  0.5172413793103449  is  0.9999999999999998
Amplification factor for sigma =  0.6206896551724138  is  0.9999999999999998
Amplification factor for sigma =  0.7241379310344828  is  0.9999999999999999
Amplification factor for sigma =  0.8275862068965517  is  0.9999999999999998
Amplification factor for sigma =  0.9310344827586207  is  0.9999999999999998
Amplification factor for sigma =  1.0344827586206897  is  0.9999999999999999
Amplification factor for sigma =  1.1379310344827587  is  1.0000000000000002
Amplification factor for sigma =  1.2413793103448276  is  1.0
Amplification factor for sigma =  1.3448275862068966  is  1.0
Amplifica